In [1]:
from pathlib import Path
from vectoria_lib.common.paths import TEST_DIR
from docx import Document

document = Document(TEST_DIR / "data/docx/2.docx")

2024-09-19 12:32:02,896 - common - INFO - Module common initialized - __init__.py - 9


In [2]:
vars(document.sections[0].header.paragraphs[0])

{'_parent': <docx.section._Header at 0x7250902f2260>,
 '_p': <CT_P '<w:p>' at 0x7250917172e0>,
 '_element': <CT_P '<w:p>' at 0x7250917172e0>}

In [3]:
vars(document.paragraphs[0])

{'_parent': <docx.document._Body at 0x7250902f2680>,
 '_p': <CT_P '<w:p>' at 0x7250917880e0>,
 '_element': <CT_P '<w:p>' at 0x7250917880e0>}

In [4]:
document.tables

[]

In [22]:

# Extract headers and footers
def extract_headers_footers(doc):
    for section in doc.sections:
        header = section.header
        footer = section.footer
        print("Header: ", end="")
        for paragraph in header.paragraphs:
            print(paragraph.text)

        print("Footer: ", end="")
        for paragraph in footer.paragraphs:
            print(paragraph.text)

In [58]:
! pip install pypandoc_binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 6.8 MB/s eta 0:00:00m eta 0:00:010:00:01


In [59]:
! python3 -m pypandoc --help

/home/leobaro/workspace/labs/eurocc/ecc-env/bin/python3: No module named pypandoc.__main__; 'pypandoc' is a package and cannot be directly executed


In [48]:
! pip install beautifulsoup4

In [49]:
import pypandoc
from bs4 import BeautifulSoup


In [63]:
# Extract Table of Contents (TOC)
def extract_toc(doc):    
    # Convert Word document to HTML
    output = pypandoc.convert_file(doc, 'html')
    
    # Parse the HTML to find the TOC section
    soup = BeautifulSoup(output, 'html.parser')
    print(soup)
    # Example: Search for <div> or <p> tags that might contain TOC
    for toc in soup.find_all('p'):
        if 'TOC' in toc.text:
            print(toc.text)

    return toc

In [66]:
print("\nExtracting Table of Contents (TOC)...")
extract_toc(TEST_DIR / "data/docx/2.docx")


Extracting Table of Contents (TOC)...
<p><a href="#first-chapter"><strong>First chapter</strong></a>
<strong>1</strong></p>
<p><a href="#_heading="><strong>Second Chapter</strong></a>
<strong>1</strong></p>
<blockquote>
<p><a href="#image">Image</a> 2</p>
</blockquote>
<h1 id="first-chapter">First chapter</h1>
<p>🥰 First Lorem ipsum</p>
<h1 id="second-chapter">Second Chapter</h1>
<p>Second Lorem ipsum</p>
<p><span class="math inline"><em>f</em>(<em>y</em>) = <em>x</em> * 2</span></p>
<table>
<colgroup>
<col style="width: 50%"/>
<col style="width: 50%"/>
</colgroup>
<thead>
<tr class="header">
<th>Lorem 1 ipsum 1</th>
<th>Lorem 1 ipsum 2</th>
</tr>
<tr class="odd">
<th>Lorem 2 ipsum 1</th>
<th>Lorem 2 ipsum 2</th>
</tr>
</thead>
<tbody>
</tbody>
</table>
<h3 id="image">Image</h3>
<p><img alt="Go away" src="media/image2.jpg" style="width:6.21875in;height:4.14583in"/></p>



<p><img alt="Go away" src="media/image2.jpg" style="width:6.21875in;height:4.14583in"/></p>

In [51]:
# A helper function to determine the level of a heading based on style
def get_heading_level(paragraph):
    if paragraph.style.name.startswith("Heading"):
        return int(paragraph.style.name.split()[-1])  # Extract the heading level (e.g., Heading 1 -> 1)
    return None  # Not a heading

# Extract paragraphs with style and hierarchy
def extract_paragraphs_with_hierarchy(doc):
    hierarchy = []  # To keep track of the hierarchy

    for paragraph in doc.paragraphs:
        style = paragraph.style.name

        # Determine if the paragraph is a heading
        heading_level = get_heading_level(paragraph)

        if heading_level:
            hierarchy.append((heading_level, paragraph.text, style))
        else:
            # If it's not a heading, assume it's normal content
            hierarchy.append((None, paragraph.text, style))

    return hierarchy

# Function to print hierarchy with indentation for better visualization
def print_hierarchy(hierarchy):
    for item in hierarchy:
        level, text, style = item
        if level is None:
            # Print normal content with default indentation
            print(f"   Content (Style: {style}): {text}")
        else:
            # Indent heading according to its level
            indent = "    " * (level - 1)
            print(f"{indent}Heading {level} (Style: {style}): {text}")


In [25]:
# Extract tables
def extract_tables(doc):
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                print(cell.text, end="\t")
            print()

In [26]:
# Extract images
def extract_images(doc):
    rels = doc.part.rels
    for rel in rels:
        if "image" in rels[rel].target_ref:
            image_part = rels[rel].target_part
            image_data = image_part.blob
            image_filename = rels[rel].target_ref.split("/")[-1]
            with open(image_filename, "wb") as img_file:
                img_file.write(image_data)
                print(f"Saved image: {image_filename}")

In [27]:
# Extract equations (as plain text since python-docx cannot handle MathML)
def extract_equations(doc):
    for paragraph in doc.paragraphs:
        if any(run.font.size == Pt(16) for run in paragraph.runs):  # Arbitrary size check for equations
            print("Equation:", paragraph.text)

In [28]:
import docx
from docx.shared import Pt

In [30]:

# Run extraction functions
print("Extracting Headers and Footers...")
extract_headers_footers(document)

Extracting Headers and Footers...
Header: Lorem ipsum header
Footer: Lorem ipsum footer




Extracting Table of Contents (TOC)...


<p><img alt="Go away" src="media/image2.jpg" style="width:6.21875in;height:4.14583in"/></p>

In [45]:
print("Extracting Paragraphs with Style and Hierarchy...")
hierarchy = extract_paragraphs_with_hierarchy(document)
print_hierarchy(hierarchy)

Extracting Paragraphs with Style and Hierarchy...
   Content (Style: Title): Test document 2
Heading 1 (Style: Heading 1): 
Heading 1 (Style: Heading 1): First chapter
   Content (Style: Normal): 🥰 First Lorem ipsum
Heading 1 (Style: Heading 1): Second Chapter
   Content (Style: Normal): Second Lorem ipsum 
   Content (Style: Normal): 
   Content (Style: Normal): 
   Content (Style: Normal): 
   Content (Style: Normal): 
        Heading 3 (Style: Heading 3): Image
   Content (Style: Normal): 


In [37]:
print("\nExtracting Tables...")
extract_tables(document)


Extracting Tables...


In [41]:
print("\nExtracting Images...")
extract_images(document)


Extracting Images...
Saved image: image2.jpg


In [40]:
print("\nExtracting Equations...")
extract_equations(document)


Extracting Equations...
